<a href="https://colab.research.google.com/github/viralka/satellite-image-processing/blob/main/test_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pystac-client sat-search 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 KB 14.7 MB/s eta 0:00:00
  Created wheel for sat-search: filename=sat_search-0.3.0-py3-none-any.whl size=9256 sha256=8a461d42c5ca8f8e12205478c2a1a6cc5b692e5e911c450df883d9cc88f57f44
  Stored in directory: /root/.cache/pip/wheels/1f/df/6e/dd93a35102c164e530a509504c924c0e9969449f7eb88befa7
  Created wheel for sat-stac: filename=sat_stac-0.4.1-py3-none-any.whl size=15199 sha256=a55dd07fb0e89aa83d49084e80227e77f071a223c895ce6ce450453a93f708aa
  Stored in directory: /root/.cache/pip/wheels/53/4a/73/9b473cab35f8a2e770ee868813194b8f6d38e393ef467a7125
Successfully built sat-search sat-stac
  Attempting 

In [ ]:
from pystac_client import Client

LandsatSTAC = Client.open("https://landsatlook.usgs.gov/stac-server", headers=[])

for collection in LandsatSTAC.get_collections():
    print(collection)

<CollectionClient id=landsat-c2l2-sr>
<CollectionClient id=landsat-c2l2-st>
<CollectionClient id=landsat-c2ard-st>
<CollectionClient id=landsat-c2l2alb-bt>
<CollectionClient id=landsat-c2l3-fsca>
<CollectionClient id=landsat-c2ard-bt>
<CollectionClient id=landsat-c2l1>
<CollectionClient id=landsat-c2l3-ba>
<CollectionClient id=landsat-c2l2alb-st>
<CollectionClient id=landsat-c2ard-sr>
<CollectionClient id=landsat-c2l2alb-sr>
<CollectionClient id=landsat-c2l2alb-ta>
<CollectionClient id=landsat-c2l3-dswe>
<CollectionClient id=landsat-c2ard-ta>


In [ ]:
import satsearch

SentinelSTAC = satsearch.Search.search( url = "https://earth-search.aws.element84.com/v0" )
print("Found " + str(SentinelSTAC.found()) + "items")

Found 56989335items


In [ ]:
def BuildSquare(lon, lat, delta):
    c1 = [lon + delta, lat + delta]
    c2 = [lon + delta, lat - delta]
    c3 = [lon - delta, lat - delta]
    c4 = [lon - delta, lat + delta]
    geometry = {"type": "Polygon", "coordinates": [[ c1, c2, c3, c4, c1 ]]}
    return geometry

geometry = BuildSquare(-59.346271, -34.233076, 0.04)
timeRange = '2019-06-01/2021-06-01'
geometry

{'type': 'Polygon',
 'coordinates': [[[-59.306271, -34.193076],
   [-59.306271, -34.273075999999996],
   [-59.386271, -34.273075999999996],
   [-59.386271, -34.193076],
   [-59.306271, -34.193076]]]}

In [ ]:
LandsatSearch = LandsatSTAC.search ( 
    intersects = geometry,
    datetime = timeRange,
    collections = ["landsat-c2l2-sr"],
    query={"eo:cloud_cover":{"lt":10}} )

Landsat_items = [i.to_dict() for i in LandsatSearch.get_items()]
print(f"{len(Landsat_items)} Landsat scenes fetched")

82 Landsat scenes fetched


In [ ]:
for item in Landsat_items:
    red_href = item['assets']['red']['href']
    red_s3 = item['assets']['red']['alternate']['s3']['href']
    print(red_href)    
    print(red_s3)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210527_20210622_02_T1/LE07_L2SP_226084_20210527_20210622_02_T1_SR_B3.TIF
s3://usgs-landsat/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210527_20210622_02_T1/LE07_L2SP_226084_20210527_20210622_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210512_20210524_02_T1/LC08_L2SP_225084_20210512_20210524_02_T1_SR_B4.TIF
s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2021/225/084/LC08_L2SP_225084_20210512_20210524_02_T1/LC08_L2SP_225084_20210512_20210524_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210511_20210606_02_T1/LE07_L2SP_226084_20210511_20210606_02_T1_SR_B3.TIF
s3://usgs-landsat/collection02/level-2/standard/etm/2021/226/084/LE07_L2SP_226084_20210511_20210606_02_T1/LE07_L2SP_226084_20210511_20210606_02_T1_SR_B3.TIF


In [ ]:
SentinelSearch = satsearch.Search.search( 
    url = "https://earth-search.aws.element84.com/v0",
    intersects = geometry,
    datetime = timeRange,
    collections = ['sentinel-s2-l2a-cogs'] )

Sentinel_items = SentinelSearch.items()
print(Sentinel_items.summary())

for item in Sentinel_items:
    red_s3 = item.assets['B04']['href']
    print(red_s3)

Items (143):
date                      id                        
2021-05-28                S2B_21HTC_20210528_0_L2A  
2021-05-23                S2A_21HTC_20210523_0_L2A  
2021-05-18                S2B_21HTC_20210518_0_L2A  
2021-05-13                S2A_21HTC_20210513_0_L2A  
2021-05-08                S2B_21HTC_20210508_0_L2A  
2021-05-03                S2A_21HTC_20210503_0_L2A  
2021-04-28                S2B_21HTC_20210428_0_L2A  
2021-04-23                S2A_21HTC_20210423_0_L2A  
2021-04-18                S2B_21HTC_20210418_0_L2A  
2021-04-13                S2A_21HTC_20210413_0_L2A  
2021-04-08                S2B_21HTC_20210408_0_L2A  
2021-04-03                S2A_21HTC_20210403_0_L2A  
2021-03-29                S2B_21HTC_20210329_0_L2A  
2021-03-24                S2A_21HTC_20210324_0_L2A  
2021-03-19                S2B_21HTC_20210319_0_L2A  
2021-03-14                S2A_21HTC_20210314_0_L2A  
2021-03-09                S2B_21HTC_20210309_0_L2A  
2021-03-04                S2A_21H

In [ ]:
item = Sentinel_items[0]
print(item.assets.keys())

dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])


In [ ]:
# %pip install rasterio[s3]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
%pip install awscli --ignore-installed six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 KB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 40.9 MB/s eta 0:00:00
  Using cached botocore-1.29.59-py3-none-any.whl (10.4 MB)
  Using cached s3transfer-0.6.0-py3-none-any.whl (79 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 20.8 MB/s eta 0:00:00
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
sat-stac 0.4.1 requires python-da

In [ ]:
# import os
# import boto3
# import rasterio as rio

# os.environ['CURL_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'

# print("Creating AWS Session")
# aws_session = rio.session.AWSSession(boto3.Session(), requester_pays=True)


In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.3 MB/s eta 0:00:00


In [ ]:
from pyproj import Transformer

def getSubset(geotiff_file, bbox):
    with rio.Env(aws_session):
        with rio.open(geotiff_file) as geo_fp:
            # Calculate pixels with PyProj 
            Transf = Transformer.from_crs("epsg:4326", geo_fp.crs) 
            lat_north, lon_west = Transf.transform(bbox[3], bbox[0])
            lat_south, lon_east = Transf.transform(bbox[1], bbox[2]) 
            x_top, y_top = geo_fp.index( lat_north, lon_west )
            x_bottom, y_bottom = geo_fp.index( lat_south, lon_east )
            # Define window in RasterIO
            window = rio.windows.Window.from_slices( ( x_top, x_bottom ), ( y_top, y_bottom ) )                
            # Actual HTTP range request
            subset = geo_fp.read(1, window=window)
    return subset

In [ ]:
def plotNDVI(nir,red,filename):
    ndvi = (nir-red)/(nir+red)
    ndvi[ndvi>1] = 1
    plt.imshow(ndvi)
    plt.savefig(filename)
    plt.close()

In [ ]:
from rasterio.features import bounds
import matplotlib.pyplot as plt

bbox = bounds(geometry)

for i,item in enumerate(Sentinel_items):
    red_s3 = item.assets['B04']['href']
    nir_s3 = item.assets['B08']['href']
    date = item.properties['datetime'][0:10]
    print("Sentinel item number " + str(i) + "/" + str(len(Sentinel_items)) + " " + date)
    red = getSubset(red_s3, bbox)
    nir = getSubset(nir_s3, bbox)
    plotNDVI(nir,red,"sentinel/" + date + "_ndvi.png")

for i,item in enumerate(Landsat_items):
    red_s3 = item['assets']['red']['alternate']['s3']['href']
    nir_s3 = item['assets']['nir08']['alternate']['s3']['href']
    date = item['properties']['datetime'][0:10]
    print("Landsat item number " + str(i) + "/" + str(len(Landsat_items)) + " " + date)
    red = getSubset(red_s3, bbox)
    nir = getSubset(nir_s3, bbox)
    plotNDVI(nir,red,"landsat/" + date + "_ndvi.png")

Sentinel item number 0/143 2021-05-28


NameError: ignored